# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [2]:
from utility import * 

# $H_4$
To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [5]:
h4 = get_qubit_hamiltonian(mol='h4', geometry=85, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(h4)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
0.1895238150168329 [Z0] +
0.08254029550391208 [Z0 Z2] +
0.08478266177100145 [Z0 Z4] +
0.09761833645634976 [Z0 Z6] +
-0.014277925412794486 [X1 Z2 X3 X5 Z6 X7] +
0.09432480368691577 [Z2 Z4] +
-0.007757209496477228 [Z4] +
0.08149422168047316 [Z4 Z6]

Fragments 2:
0.03302178088935298 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7]

Number of fragments: 37


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_4$ by partitioning its terms into mutually commuting fragments. 

In [6]:
comm_groups = get_commuting_group(h4)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 4
The first commuting group
-0.6362710050108029 [] +
-0.03245891907320156 [X0 X1 Y4 Y5] +
-0.020882678880093687 [X0 X1 Y6 Y7] +
-0.011726041549656178 [X0 Z1 Y2 Y4 Z5 X6] +
0.03302178088935298 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
0.021295739339696803 [X0 Z1 Z2 X3 X5 X6] +
-0.03245891907320156 [Y0 Y1 X4 X5] +
-0.020882678880093687 [Y0 Y1 X6 X7] +
-0.011726041549656178 [Y0 Z1 X2 X4 Z5 Y6] +
0.03302178088935298 [Y0 Z1 Z2 Y3 X4 Z5 Z6 X7] +
0.021295739339696803 [Y0 Z1 Z2 Y3 Y5 Y6] +
0.021295739339696803 [X1 X2 X4 Z5 Z6 X7] +
0.03302178088935298 [X1 X2 Y5 Y6] +
-0.011726041549656178 [X1 Z2 Y3 Y5 Z6 X7] +
0.021295739339696803 [Y1 Y2 Y4 Z5 Z6 Y7] +
0.03302178088935298 [Y1 Y2 X5 X6] +
-0.011726041549656178 [Y1 Z2 X3 X5 Z6 Y7] +
-0.021728632837408966 [X2 X3 Y4 Y5] +
-0.03617627181461982 [X2 X3 Y6 Y7] +
-0.021728632837408966 [Y2 Y3 X4 X5] +
-0.03617627181461982 [Y2 Y3 X6 X7]


To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [7]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999999+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [8]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-0.6362710050107999 [] +
-0.021295739339696723 [X0 X1 X2 Z3 Z5 Z6] +
-0.03302178088935293 [X0 X1 X2 Z3 Z7] +
0.0324589190732016 [X0 X1 Z5] +
0.0117260415496561 [X0 X1 Z6] +
-0.020882678880093743 [X0 X1 Z6 Z7] +
-0.03302178088935293 [X1 X2 Z3 X4 Z7] +
-0.021295739339696723 [X1 X2 Z3 Z5 Z6] +
-0.0117260415496561 [X1 X2 Z3 Z5 Z7] +
0.0117260415496561 [X1 X2 X4 Z6] +
0.021295739339696723 [X1 X2 X4 Z7] +
0.03302178088935293 [X1 X2 Z5 Z6] +
0.021728632837408914 [X2 Z3 Z5] +
-0.0117260415496561 [X2 Z3 Z5 Z7] +
-0.03617627181461987 [X2 Z3 Z6 Z7] +
0.021728632837408914 [Z3 X4 Z5] +
-0.03617627181461987 [Z3 Z6 Z7] +
0.0324589190732016 [X4 Z5] +
0.03302178088935293 [Z5 Z6] +
-0.020882678880093743 [Z6 Z7] +
0.021295739339696723 [Z7]


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [9]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999996 []

The all-z form of qwc fragment:
-0.6362710050107995 [] +
-0.021295739339696702 [Z0 Z1 Z2 Z3 Z5 Z6] +
-0.03302178088935291 [Z0 Z1 Z2 Z3 Z7] +
0.03245891907320158 [Z0 Z1 Z5] +
0.011726041549656093 [Z0 Z1 Z6] +
-0.020882678880093725 [Z0 Z1 Z6 Z7] +
-0.03302178088935291 [Z1 Z2 Z3 Z4 Z7] +
-0.021295739339696702 [Z1 Z2 Z3 Z5 Z6] +
-0.011726041549656093 [Z1 Z2 Z3 Z5 Z7] +
0.011726041549656093 [Z1 Z2 Z4 Z6] +
0.021295739339696702 [Z1 Z2 Z4 Z7] +
0.03302178088935291 [Z1 Z2 Z5 Z6] +
0.021728632837408893 [Z2 Z3 Z5] +
-0.011726041549656093 [Z2 Z3 Z5 Z7] +
-0.03617627181461984 [Z2 Z3 Z6 Z7] +
0.021728632837408893 [Z3 Z4 Z5] +
-0.03617627181461984 [Z3 Z6 Z7] +
0.03245891907320158 [Z4 Z5] +
0.03302178088935291 [Z5 Z6] +
-0.020882678880093725 [Z6 Z7] +
0.021295739339696702 [Z7]
